In [1]:
import networkx as nx
import json
from textblob import TextBlob
from datetime import datetime
from community import community_louvain
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = []
json_data=open('path\\Romaine.json').read()
data.append(json.loads(json_data))


In [3]:

UserData = {}

for l in range(len(data)):
    for i in range(len(data[l])):
        for j in range(len(data[l][i])):
            obj = data[l][i][j]
            hashtags = {}
            if 'user' in obj:
                testimonial = TextBlob(obj['full_text'])
                if obj['user']['screen_name'] in UserData:
                    hashtags = UserData[obj['user']['screen_name']]
                for h in obj['entities']['hashtags']:
                    if not '#' + h['text'] in hashtags:
                        hashtags['#' + h['text']] = {}
                        hashtags['#' + h['text']]['tweeted'] = obj['created_at']
                        hashtags['#' + h['text']]['sentiment'] = testimonial.sentiment
                UserData[obj['user']['screen_name']] = hashtags

In [4]:
i = 0
for user in UserData:
    if i == 30:
        break
    print(user)
    print(json.dumps(UserData[user],indent=1))
    i += 1

mward04
{
 "#Turkey": {
  "tweeted": "Wed Nov 21 01:22:46 +0000 2018",
  "sentiment": [
   0.625,
   0.5
  ]
 },
 "#Romaine": {
  "tweeted": "Wed Nov 21 01:22:46 +0000 2018",
  "sentiment": [
   0.625,
   0.5
  ]
 },
 "#Thanksgiving": {
  "tweeted": "Wed Nov 21 01:22:46 +0000 2018",
  "sentiment": [
   0.625,
   0.5
  ]
 },
 "#SaidNoOneEver": {
  "tweeted": "Wed Nov 21 01:22:46 +0000 2018",
  "sentiment": [
   0.625,
   0.5
  ]
 }
}
Johnny_Solo
{
 "#Romaine": {
  "tweeted": "Wed Nov 21 01:22:38 +0000 2018",
  "sentiment": [
   0.3,
   0.5
  ]
 },
 "#Salad": {
  "tweeted": "Wed Nov 21 01:22:38 +0000 2018",
  "sentiment": [
   0.3,
   0.5
  ]
 },
 "#Yummy": {
  "tweeted": "Wed Nov 21 01:22:38 +0000 2018",
  "sentiment": [
   0.3,
   0.5
  ]
 }
}
l8teebug
{
 "#romaine": {
  "tweeted": "Wed Nov 21 01:20:24 +0000 2018",
  "sentiment": [
   -0.2,
   0.39999999999999997
  ]
 }
}
mailtribune
{
 "#ALERT": {
  "tweeted": "Wed Nov 21 01:20:04 +0000 2018",
  "sentiment": [
   0.13636363636363635,


In [5]:
def getSentimentArea(sentiment):
    if sentiment[0] == 0 and sentiment[1] == 0:
        return 1
    if sentiment[0] == 0 and sentiment[1] > 0:
        return 2
    if sentiment[0] > 0 and sentiment[1] > 0:
        return 3
    if sentiment[0] > 0 and sentiment[1] == 0:
        return 4
    if sentiment[0] == 0 and sentiment[1] < 0:
        return 5
    if sentiment[0] < 0 and sentiment[1] < 0:
        return 6
    if sentiment[0] < 0 and sentiment[1] == 0:
        return 7
    if sentiment[0] < 0 and sentiment[1] > 0:
        return 8
    if sentiment[0] > 0 and sentiment[1] < 0:
        return 9
    
    return 10

In [6]:
def calcHashtagWeight(hashtag1,hashtag2):
    fmt = '%a %b %d %H:%M:%S +0000 %Y'
    d1 = datetime.strptime(hashtag1['tweeted'], fmt)
    d2 = datetime.strptime(hashtag2['tweeted'], fmt)

    minutesDiff = 20000.0 - ((d1 - d2).seconds/60.0)
    
    s1 = getSentimentArea(hashtag1['sentiment'])
    s2 = getSentimentArea(hashtag2['sentiment'])
    
    if s1!=s2: #if the sentiment area is different then consider not connected. Optionally we could reduce the weight by a factor
        minutesDiff = 0
        
    return minutesDiff

In [7]:
def calcWeight(user1,user2):
    weight=0
    for hashtag in user1:
        if hashtag in user2:
            weight+=calcHashtagWeight(user1[hashtag],user2[hashtag])
    return weight

In [14]:
gephiData = []
i=-1
j=-1
for user1 in UserData:
    i=i+1
    j=-1
    if i%100==0:
        print("Processing please wait ... {0}%".format(i*100.0/len(UserData)))
    for user2 in UserData:
        j=j+1
        if j>i:
            weight = calcWeight(UserData[user1],UserData[user2])
            if weight > 0:
                edge = []
                edge.append(user1)
                edge.append(user2)
                edge.append('undirected')
                edge.append(weight)
                gephiData.append(edge)

np.savetxt("path\\Romaine_edges.csv", gephiData, delimiter=",", fmt='%s', header="Source,Target,Type,Weight",comments='')
print('Done')

Processing please wait ... 0.0%
Processing please wait ... 15.97444089456869%
Processing please wait ... 31.94888178913738%
Processing please wait ... 47.92332268370607%
Processing please wait ... 63.89776357827476%
Processing please wait ... 79.87220447284345%
Processing please wait ... 95.84664536741214%


In [26]:
def getTweets(username):
    for l in range(len(data)):
        for i in range(len(data[l])):
            for j in range(len(data[l][i])):
                obj = data[l][i][j]
                if 'user' in obj:
                    if obj['user']['screen_name'] == username:
                        print("User: {0} Tweet: {1}".format(username,obj['full_text']))
                    